# 1. Blocker Fraud Company
 <font size="2"> Dados e problema negócio hipotéticos</font>
 
A Blocker Fraud Company é uma empresa especializada na detecção de fraudes em transações financeiras feitas atravś de dispositivos móveis.

O modelo de negócio da empresa é do tipo Serviço com a monetização feita por performance do serviço prestado, ou seja, o usuário paga uma taxa fixa sobre o sucesso da detecção de fraude das transações.

## 1.1. Estratégia de expansão

A Blocker Fraud Company está em fase de expansão no Brasil e para adquirir clientes mais rapidamente, ela adotou uma estratégia muito agressiva. A estratégia funciona da seguinte forma:

1. A empresa vai receber 25% do valor de cada transação detectada verdadeiramente como fraude.
2. A empresa vai receber 5% do valor de cada transação detectada como fraude, porém a transação é verdadeiramente legítima.
3. A empresa vai devolver 100% do valor para o cliente, a cada transação detectada como legítima, porém a transação é verdadeiramente uma fraude.

Com essa estratégia agressiva a empresa assume os riscos em falhar na detecção de fraude e é remunerada na detecção assertiva das fraudes.

## 1.2. Objetivo

Criar um modelo de alta precisão e acurácia na detecção de fraudes de transações feitas através de dispositivos móveis.

### 1.2.1. Entregáveis

Modelo em produção no qual seu acesso será feito via API, ou seja, os clientes enviarão suas transações via API para que o seu modelo as classifique como fraudulentas ou legítimas.

Além disso, você precisará entregar um relatório reportando a performance e os resultados do seu modelo em relação ao lucro e prejuízo que a empresa terá ao usar o modelo que você produziu. No seu relatório deve conter as respostas para as seguintes perguntas:

1. Qual a Precisão e Acurácia do modelo?
2. Qual a Confiabilidade do modelo em classificar as transações como legítimas ou fraudulentas?
3. Qual o Faturamento Esperado pela Empresa se classificarmos 100% das transações com o modelo?
4. Qual o Prejuízo Esperado pela Empresa em caso de falha do modelo?
5. Qual o Lucro Esperado pela Blocker Fraud Company ao utilizar o modelo?


# 2. Imports

In [1]:
import joblib
import pandas as pd
import numpy as np

## 2.1. Utils functions

In [39]:
def calculate_value_transaction_as_fraud(percentage: float, df:pd.DataFrame) -> float:
    """
        Cálculo da: empresa vai receber 25% do valor de cada transação detectada verdadeiramente como fraude.

        Parameters
        ----------
        percentage: float 
        df: Pandas DataFrame

        Returns
        -------
        value_to_receive: float 
            Valor calculado que a empresa vai receber com as fraudes detectadas
    """
    fraud_detected = df[(df['isFraud'] == 1) & (df['predictions'] == 1)]
    fraud_detected_amount = fraud_detected[['amount', 'isFraud', 'predictions']].groupby(['isFraud', 'predictions']).sum().reset_index()
    fraud_detected_amount['to_receive'] = fraud_detected_amount['amount']*percentage
    value_to_receive = fraud_detected_amount['to_receive'][0]
    return value_to_receive


def calculate_legitimate_as_fraud(percentage: float, df:pd.DataFrame) -> float:
    """
        Cálculo da: A empresa vai receber 5% do valor de cada transação detectada como fraude, porém a transação é verdadeiramente legítima.

        Parameters
        ----------
        percentage: float 
        df: Pandas DataFrame

        Returns
        -------
        value_to_receive: float 
            Valor calculado que a empresa vai receber com as fraudes legítimas detectadas
    """
    fraud_detected_leg = df[(df['isFraud'] == 0) & (df['predictions'] == 1)]
    fraud_detected_leg_amount = fraud_detected_leg[['amount', 'isFraud', 'predictions']].groupby(['isFraud', 'predictions']).sum().reset_index()
    fraud_detected_leg_amount['to_receive'] = fraud_detected_leg_amount['amount']*percentage
    value_to_receive = fraud_detected_leg_amount['to_receive'][0]
    return value_to_receive

def calculate_refund_to_customer(df:pd.DataFrame) -> float:
    """
        Cálculo da: A empresa vai devolver 100% do valor para o cliente, a cada transação detectada como legítima, porém a transação é verdadeiramente uma fraude.
        Parameters
        ----------
        df: Pandas DataFrame

        Returns
        -------
        value_refund: float 
            Valor calculado que a empresa vai devolver ao cliente
    """

    fraud_not_detected = df[(df['isFraud'] == 1) & (df['predictions'] == 0)]
    fraud_not_detected_amount = fraud_not_detected[['amount', 'isFraud', 'predictions']].groupby(['isFraud', 'predictions']).sum().reset_index()
    value_refund = fraud_not_detected_amount['amount'][0]
    return value_refund

def create_features(df: pd.DataFrame) -> pd.DataFrame:
    """
        Cria as novas features a partir dos dados de entrada
        Parameters
        ----------
        df: Pandas DataFrame

        Returns
        -------
        df: Pandas DataFrame
            Novo dataframe com as features inseridas.
    """
    df['is_high_amount'] = define_high_amount(df, 179000)
    df['is_M_customer'] = define_merchant_customer(df)
    df['is_new_balance_orig_zero'] =  define_is_balance_zero(df, 'newbalanceOrig')
    df['is_new_balance_dest_zero'] = define_is_balance_zero(df, 'newbalanceDest')
    df['diff_orig'] = difference_between_two_columns(df, 'oldbalanceOrg', 'newbalanceOrig')
    df['diff_dest'] = difference_between_two_columns(df, 'oldbalanceDest', 'newbalanceDest')
    df['is_orig_old_balance_greater_new'] = define_column_is_greater(df, 'oldbalanceOrg','newbalanceOrig')
    df['is_dest_old_balance_greater_new'] = define_column_is_greater(df, 'oldbalanceDest','newbalanceDest')
    return df

def define_high_amount(df: pd.DataFrame, value: int) -> np.array:
    is_high_amount = np.where(df['amount']>=value, 1, 0)
    return is_high_amount

def define_merchant_customer(df: pd.DataFrame) -> np.array:
    is_merchant_customer = np.where(df['nameDest'].str.contains('M'), 1, 0)
    return is_merchant_customer

def define_is_balance_zero(df: pd.DataFrame, column: str) -> np.array:
    is_column_balance_zero = np.where(df[column]==0, 1, 0)
    return is_column_balance_zero

def difference_between_two_columns(df: pd.DataFrame, left_column: str, right_column: str) -> pd.Series:
    difference_result = df[left_column]-df[right_column]
    return difference_result

def define_column_is_greater(df: pd.DataFrame, left_column: str, right_column: str) -> np.array:
    is_greater_result = np.where(df[left_column]>df[right_column], 1, 0)
    return is_greater_result

def define_final_data_to_model(df: pd.DataFrame) -> pd.DataFrame:
    cols_to_include = ['isFlaggedFraud', 'is_high_amount', 'is_M_customer', 
                   'is_new_balance_orig_zero', 'is_orig_old_balance_greater_new',
                   'is_new_balance_dest_zero', 'is_dest_old_balance_greater_new']

    columns_drop = ['nameOrig', 'nameDest']

    for col in cols_to_include:
        df[col] = df[col].astype('category')

    df.drop(columns_drop, axis=1, inplace=True)
    return df

def predict(data, model):   
    predicted = model.predict(data)    
    return int(predicted[0])

# 3. Load model

In [4]:
model = joblib.load('../models/model_cicle_1.pkl')

In [5]:
model

Pipeline(steps=[('enconding_rare',
                 RareLabelEncoder(variables=['type', 'isFlaggedFraud'])),
                ('enconding_cat',
                 CountFrequencyEncoder(variables=['is_new_balance_dest_zero',
                                                  'type',
                                                  'is_orig_old_balance_greater_new',
                                                  'is_M_customer',
                                                  'is_high_amount',
                                                  'is_new_balance_orig_zero',
                                                  'is_dest_old_balance_greater_new',
                                                  'isFlaggedFraud'])),
                ('enconding_num',
                 SklearnTransformerWrapper(transformer=StandardScaler(),
                                           variables=['amount', 'oldbalanceOrg',
                                                      'newbalanceOrig',
       

# 4. Load test dataset

In [21]:
df_test = pd.read_parquet('../data/raw/df_applied_features_test.parquet.gzip') 
df_test_y_real = df_test.isFraud.values
df_test.drop('isFraud', axis=1, inplace=True)

In [22]:
df_test = define_final_data_to_model(create_features(df_test))
df_test.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud,is_high_amount,is_M_customer,is_new_balance_orig_zero,is_new_balance_dest_zero,diff_orig,diff_dest,is_orig_old_balance_greater_new,is_dest_old_balance_greater_new
3724554,278,PAYMENT,16517.82,4621.74,0.00,0.00,0.00,0,0,1,1,1,4621.74,0.00,1,0
2007406,179,CASH_OUT,288543.05,0.00,0.00,552863.94,841406.99,0,1,0,1,0,0.00,-288543.05,0,0
38581,9,PAYMENT,39195.58,21818.00,0.00,0.00,0.00,0,0,1,1,1,21818.00,0.00,1,0
2332625,188,PAYMENT,2006.64,47440.84,45434.20,0.00,0.00,0,0,1,0,1,2006.64,0.00,1,0
3572454,261,CASH_IN,233785.38,160279.00,394064.38,3026936.33,2793150.95,0,1,0,0,0,-233785.38,233785.38,0,1


# 5. Predict data

In [23]:
y_pred = model.predict(df_test) 

# 6. Business performance

In [44]:
df_test['predictions'] = y_pred
df_test['isFraud'] = df_test_y_real

to_receive = calculate_value_transaction_as_fraud(0.25, df_test)
to_receive_legit = calculate_legitimate_as_fraud(0.05, df_test)
to_refund = calculate_refund_to_customer(df_test)

print('A empresa vai receber {:,.2f} devido a transações realmente detectadas como fraude'.format(to_receive))
print('A empresa vai receber {:,.2f} devido a transações detectadas como fraude, mas realmente legítimas'.format(to_receive_legit))
print('A empresa vai devolver {:,.2f} devido a transações detectadas como legítimas, mas na verdade era fraude'.format(to_refund))

A empresa vai receber 32,536,066.74 devido a transações realmente detectadas como fraude
A empresa vai receber 45,439.13 devido a transações detectadas como fraude, mas realmente legítimas
A empresa vai devolver 2,846,573.25 devido a transações detectadas como legítimas, mas na verdade era fraude


# 7. Conclusion

Dado a finalização desse ciclo e também o cálculo da performance que o modelo impactará no negócio, podemos então partir para os entregáveis requisitado pela empresa, eles são:

1. Qual a Precisão e Acurácia do modelo?
2. Qual a Confiabilidade do modelo em classificar as transações como legítimas ou fraudulentas?
3. Qual o Faturamento Esperado pela Empresa se classificarmos 100% das transações com o modelo?
4. Qual o Prejuízo Esperado pela Empresa em caso de falha do modelo?
5. Qual o Lucro Esperado pela Blocker Fraud Company ao utilizar o modelo?

1. Conferindo o notebook 7-1-model-performance tivemos que o modelo alcançou uma acurácia de 99.95% e um recall de 72.68 e um f1-score de 82.42% sendo essa últimas métricas mais confiáveis do que a acurácia dado a natureza do problema de fraude e por causa dos dados serem bastante desbalanceados.
2. A confiabilidade do modelo está em 72.68% em detectar as transações fraudulentas.
3. O faturamento esperado olhando os dados de teste foi de 32 milhões.
4. O prejuízo está em torno de 2 milhões para as transações que era legítimas mas foram classificadas como fraude.
5. O lucro esperado é de 30 milhões, seria o faturamento esperado menos o prejuízo esperado.